In [3]:
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torch.optim import lr_scheduler
import copy
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import torch.optim as optim
import random
import torch.nn as nn
import torch.nn.functional as F
import pprint, pickle
from torchvision import datasets, models, transforms
import time
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image


from __future__ import print_function, division

from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy

In [4]:
"""
i:166 (* 8)
o:82  (*16)
p:1330
u:83  (*16)
""" 
def default_loader(path):
    pkl_file = open(path, 'rb')
    data1 = pickle.load(pkl_file)
    data1 = data1/256
    pkl_file.close()
    return data1
class MyDataset(Dataset):
    def __init__(self, txt, transform=None, target_transform=None, loader=default_loader):
        fh = open(txt, 'r')
        imgs = []
        for line in fh:
            line = line.strip('\n')
            line = line.rstrip()
            words = line.split()
            if str(words[1]) == "i":
                label = 0
            elif str(words[1]) == "o":
                label = 1
            elif str(words[1]) == "u":
                label = 2
            else:
                label = 3
            imgs.append((words[0],str(label)))

        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = self.loader(fn)
        img = img[256:768,256:768,:]#crop the cernter part of the image
        if self.transform is not None:
            img = self.transform(img)
        return img,label

    def __len__(self):
        return len(self.imgs)

In [5]:
data_transforms = {
    'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
#         transforms.Normalize(mean = [0.5]*30, std = [0.5]*30)
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
#         transforms.Normalize(mean = (0.5, 0.5, 0.5), std = (0.5, 0.5, 0.5))
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

trainset = MyDataset(txt='D:\\Test_Data2\\list1.txt',transform=data_transforms['train'])

train_size = 0.70
val_size = 0.20
test_size = 0.10
num_train = len(trainset)
indices = list(range(num_train))
random.shuffle(indices)
split = int(np.floor(train_size * num_train))
split_val = int(np.floor(val_size * num_train))

train_idx, valid_idx, test_idx = indices[:split], indices[split:split_val+split], indices[split_val+split:]

image_datasets={'train': MyDataset(txt='D:\\Test_Data2\\list1.txt',  transform=data_transforms['train']),
                'val': MyDataset(txt='D:\\Test_Data2\\list1.txt',  transform=data_transforms['val']),
                'test': MyDataset(txt='D:\\Test_Data2\\list1.txt',  transform=data_transforms['test']),
                }
train_sampler = torch.utils.data.sampler.RandomSampler(train_idx)
val_sampler = torch.utils.data.sampler.RandomSampler(valid_idx)
test_sampler = torch.utils.data.sampler.RandomSampler(test_idx)

dataloaders = {
        'train': torch.utils.data.DataLoader(trainset, batch_size=4,sampler = train_sampler,
                                          shuffle=False, num_workers=0),
        'val': torch.utils.data.DataLoader(trainset, batch_size=4,sampler = val_sampler,
                                          shuffle=False, num_workers=0),
        'test': torch.utils.data.DataLoader(trainset, batch_size=1,sampler = test_sampler,
                                          shuffle=False, num_workers=0),
                }
dataset_sizes = {
        'train':num_train* train_size,
        'val': num_train* val_size,
        'test': num_train * test_size
				    }

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
inputs, classes = next(iter(dataloaders['train']))
print(np.shape(inputs))
print(classes)

torch.Size([4, 30, 512, 512])
('3', '0', '0', '1')


In [11]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(30, 32, 5, stride=1, padding=2)
        self.conv1_bn = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=1)
        self.conv2_bn = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(64 * 16 * 16, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        
        #x = self.pool(F.relu(self.conv1(x)))
        #x = self.pool(F.relu(self.conv2(x)))
        #x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.conv1(x))
        x = self.conv1_bn(x)
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.conv2_bn(x)
        x = self.pool2(x)
        x = x.view(-1, 64*16*16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = Net()

use_gpu = torch.cuda.is_available()  # 
if use_gpu:
    net = net.cuda()
#    
import torch.optim as optim


#criterion =nn.BCELoss()

#criterion =nn.NLLLoss(weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='elementwise_mean')

#m= nn.Sigmoid()
#criterion = nn.NLLLoss()
#criterion=nn.BCECriterion()


criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr = 0.001, betas=(0.9, 0.999))
epc = 30
loss_mat = np.zeros(epc)
loss_V = np.zeros(epc)
accuracy_T = np.zeros(epc)
accuracy = np.zeros(epc)
j = 0
loss_mat = np.zeros(epc)
loss_V = np.zeros(epc)
trainloader = dataloaders['train']
testloader = dataloaders['val']
EPOCH = 135   #total epoches
pre_epoch = 0  # epoches existed
LR = 0.1       

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )

    def forward(self, x):
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        #change the channel to 30
        self.conv1 = nn.Sequential(
            nn.Conv2d(30, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)   #strides=[1,1]
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out


def ResNet18():

    return ResNet(ResidualBlock)
net = ResNet18()
use_gpu = torch.cuda.is_available()  # 
if use_gpu:
    net = net.cuda()
epc = 100
criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr = 0.001, betas=(0.9, 0.999))
loss_mat = np.zeros(epc)
loss_V = np.zeros(epc)
trainloader = dataloaders['train']
valloader = dataloaders['val']
EPOCH = 135   #total epoches
pre_epoch = 0  # epoches existed
LR = 0.1        


In [12]:
if __name__ == "__main__":
    best_acc = 85 
    print("Start Training, Resnet-18!") 
    with open("acc.txt", "w") as f:
        with open("log.txt", "w")as f2:
            for epoch in range(pre_epoch, EPOCH):
                print('\nEpoch: %d' % (epoch + 1))
                net.train()
                sum_loss = 0.0
                correct = 0.0
                total = 0.0
                for i, data in enumerate(trainloader, 0):
                    length = len(trainloader)
                    inputs, labels = data
#                     inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    # forward + backward
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    sum_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += predicted.eq(labels.data).cpu().sum()
                    print('[epoch:%d, iter:%d] Loss: %.03f | Acc: %.3f%% '
                          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                    f2.write('%03d  %05d |Loss: %.03f | Acc: %.3f%% '
                          % (epoch + 1, (i + 1 + epoch * length), sum_loss / (i + 1), 100. * correct / total))
                    f2.write('\n')
                    f2.flush()

                # 每训练完一个epoch测试一下准确率
                print("Waiting Test!")
                with torch.no_grad():
                    correct = 0
                    total = 0
                    for data in testloader:
                        net.eval()
                        images, labels = data
#                         images, labels = images.to(device), labels.to(device)
                        outputs = net(images)
                        
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum()
                    print('accuracy：%.3f%%' % (100 * correct / total))
                    acc = 100. * correct / total
                    # 将每次测试结果实时写入acc.txt文件中
                    print('Saving model......')
                    torch.save(net.state_dict(), '%s/net_%03d.pth' % (args.outf, epoch + 1))
                    f.write("EPOCH=%03d,Accuracy= %.3f%%" % (epoch + 1, acc))
                    f.write('\n')
                    f.flush()
                    # save result to best_acc.txt
                    if acc > best_acc:
                        f3 = open("best_acc.txt", "w")
                        f3.write("EPOCH=%d,best_acc= %.3f%%" % (epoch + 1, acc))
                        f3.close()
                        best_acc = acc
            print("Training Finished, TotalEPOCH=%d" % EPOCH)

Start Training, Resnet-18!

Epoch: 1


AttributeError: 'tuple' object has no attribute 'requires_grad'

In [ ]:
for epoch in range(epc):  # loop over the dataset multiple times
    correct = 0.0
    total = 0.0
    running_loss = 0.0
    running_loss_V = 0.0
    correct_T = 0.0
    total_T = 0.0

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
#         inputs, labels = inputs.cuda(), labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total_T += labels.size(0)
        correct_T += (predicted == labels).sum().item()
        accuracy_T[epoch] = 100 * correct_T / total_T
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000== 1999:    # print every 2000 mini-batches
            print('[%d, %5d] training loss: %.3f training accuracy: %.2f' %
                  (epoch + 1, i + 1, running_loss /  2000, accuracy_T[epoch]))
            for data in valloader:
                images, labels = data
#                 images, labels = images.cuda(), labels.cuda()
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                #print(total ,'   ', correct)
                correct += (predicted == labels).sum().item()
                accuracy[epoch] = 100 * correct / total
                loss = criterion(outputs, labels)
                running_loss_V += loss.item()
            loss_mat[epoch] = running_loss / 2000
            loss_V[epoch] = running_loss_V / (n_val_samples/4)
            print('Validation loss: %.3f validation accuracy: %.2f' %
                (loss_V[epoch] , accuracy[epoch]))
            running_loss = 0.0
            running_loss_V = 0.0
            correct = 0
            total = 0
            

print('Finished Training')